In [5]:
import random as rd
import time
import matplotlib.pyplot as plt
import math
import random
import multiprocessing as mp

# Création de la queue pour stocker les résultats
resultat_queue = mp.Queue()

**Fonctions**

In [2]:
def TakeWeight(elem):
    #On prend le poids du noeux
    return elem[2]

def Connexe(Graph):
    #On vérifie que le graphe est connexe
    #On considère que les noeux varient de 0 à n-1
    #
    noeux = {element[0] for element in Graph}
    if len(noeux) == max(noeux)+1 :
        return True
    return False

def SortGraph(Graph,filter=False):
    #On trie le graphe par ordre croissant des poids
    #Graph est donc une liste de tuples (A,B,X)
    NewGraph = []
    for element in Graph :
        if element[0] >= element[1] :
            if filter:
                #Si le filtre est activé, on ne prend pas les arêtes qui boucle avec elle même
                if element[0]==element[1] :
                    NewGraph.append(element)
            else :
                NewGraph.append((element[1],element[0],element[2]))
        else :
            NewGraph.append(element)
    NewGraph.sort(key=TakeWeight)
    return NewGraph

def PoidsGraph(Graph) :
    #print(Graph)
    return sum([element[2] for element in Graph])

def Nb_Nodes(Graph) :
    #On renvoie le nombre de noeux
    Nodes = set()
    for element in Graph :
        Nodes.add(element[0])
        Nodes.add(element[1])
    return len(Nodes)


def CheckFullGraph(Graph,Sous_Graph) :
    #On vérifie que le graphe passe par toutes les sommets
    Nb_sommets_graph = Nb_Nodes(Graph)
    Nb_sommets_sous_graph = Nb_Nodes(Sous_Graph)
    if Nb_sommets_graph == Nb_sommets_sous_graph :
        return True
    return False

def distance_euclidienne(A,B):
    #On calcule la distance euclidienne entre deux points
    return ((A[0]-B[0])**2+(A[1]-B[1])**2)**(1/2)

def random_graph_complet(nombre_noeud,zone=100) : #Resoudre l'unicité
    #On génère un graphe complet aléatoire
    Data = {}
    for i in range(nombre_noeud) :
        while True :
            NewNode = (rd.randint(0,zone),rd.randint(0,zone))
            if NewNode not in Data.values() :
                Data[i] = NewNode
                break
    return Data

def alpha_plt(Weight,Liens):
    #On calcule le coefficient alpha pour le plot
    #On suppose que Liens est une liste de tuples (A,B,X)
    poids = [element[2] for element in Liens]
    return Weight/max(poids)


def plot_graph(Graph,Liens=[],show_numbers=True):
    #On plot le graphe
    X = [element[0] for element in Graph.values()]
    Y = [element[1] for element in Graph.values()]
    plt.scatter(X,Y)
    if show_numbers :
        for i in range(len(Graph)):
            plt.text(X[i], Y[i], str(i), color='orange', fontsize=10, ha='center', va='center')



    # On plot les liens avec leurs poids
    for lien in Liens:
        node_A, node_B, weight = lien
        coord_A = Graph[node_A]
        coord_B = Graph[node_B]
        plt.plot([coord_A[0], coord_B[0]], [coord_A[1], coord_B[1]], 'k-', alpha=alpha_plt(weight,Liens))  # Lien en noir

        # Ajouter le poids du lien
        #text_x = (coord_A[0] + coord_B[0]) / 2
        #text_y = (coord_A[1] + coord_B[1]) / 2
        #plt.text(text_x, text_y, str(weight), color='red', fontsize=8, ha='center', va='center')

    plt.show()

def plot_perf(Perf,support=False):
    #On plot la performance
    # Extraction des temps et des tailles dans des listes distinctes
    temps = [t[1] for t in Perf]
    tailles = [t[0] for t in Perf]
    print(tailles)
    #on calcul le coef des support
    if support :
        Tempsmax = Perf[-1][1]
        Taillemax = Perf[-1][0]
        coeflinaire = Tempsmax/Taillemax
        coefquadratique = Tempsmax/(Taillemax**2)
        coef3 = Tempsmax/(Taillemax**3)
        coeflinlog = Tempsmax/(Taillemax*math.log(Taillemax))
        coefexponentiel = Tempsmax/(2**Taillemax)
        coeflogarithmique = Tempsmax/(math.log(Taillemax))
        print(coeflinaire,coefquadratique,coefexponentiel,coeflogarithmique)

    # Tracé de la courbe
    plt.plot(tailles,temps)  # 'o-' pour des marqueurs en forme de cercles reliés par des lignes
    if support :
        # Courbe linéaire : O(n)
        plt.plot(tailles, [coeflinaire*val for val in tailles], '--', label='O(n)', color='red')

        # Courbe quadratique : O(n^2)
        plt.plot(tailles, [coefquadratique*(val**2) for val in tailles], '--', label='O(n^2)', color='orange')

        # Courbe O(n^3)
        plt.plot(tailles, [coef3*(val**3) for val in tailles], '--', label='O(n^3)', color='yellow')

        # Courbe linéaire-logarithmique : O(n*log(n))
        plt.plot(tailles, [coeflinlog * val * math.log(val) for val in tailles], '--', label='O(n*log(n))', color='cyan')

        # Courbe exponentielle : O(2^n)
        plt.plot(tailles, [coefexponentiel*(2**val) for val in tailles], '--', label='O(2^n)', color='green')

        # Courbe logarithmique : O(log(n))
        plt.plot(tailles, [coeflogarithmique * math.log(val) for val in tailles], '--', label='O(log(n))', color='purple')


    # Configuration des labels des axes et du titre
    plt.xlabel('Taille (nombre de noeuds)')
    plt.ylabel('Temps (s)')
    plt.title('Performance en fonction de la taille')
    plt.legend()

    # Affichage du graphique
    plt.show()

def fourmi_proba_process(i,resultat_queue,Graph,Pheromone,PositionFourmis,SeenFourmis,alpha=1,beta=1) :
    #On filtre le cas ou la fourmi doit rejoindre son noeud d'origine
    Last = True
    for bool in SeenFourmis[i] :
        if bool == False :
            Last = False
    if Last :
        #On renvoie le signal qu'une fourmi doit revenir à son point de départ
        return ["F"]*len(PositionFourmis)
    Result_temp = []
    Distances_temp = []
    #On la calculer pour chaque fourmis les probabilités d'aller à chaque noeud
    #Coordonnées de la fourmis
    CoordsFourmis = Graph[PositionFourmis[i]]
    for j in range(len(Graph)) :
        #j est le noeud de destination
        #On veux pas que la fourmis reste sur place ni qu'elle aille sur un noeud qu'elle a déjà vu
        if j != PositionFourmis[i] and SeenFourmis[i][j] == False:
            Result_temp.append([PositionFourmis[i],j,(Pheromone[PositionFourmis[i]][j])**(beta)/(distance_euclidienne(CoordsFourmis,Graph[j]))**(alpha)])
            Distances_temp.append((Pheromone[PositionFourmis[i]][j])**(beta)/(distance_euclidienne(CoordsFourmis,Graph[j]))**(alpha))
        else :
            Result_temp.append((PositionFourmis[i], j, None))
            Distances_temp.append(None)
    #On return tout au thread principal
    resultat_queue.put(Result_temp,Distances_temp)



def fourmis_proba_main(Graph,Pheromone,PositionFourmis,SeenFourmis,alpha=1,beta=1) : #Trustable
    Result = []
    # /!\ On met dans Distances et Distances_temps les distances inverses multiplié par les phéromones
    Distances = []
    processes = []
    for i in range(len(PositionFourmis)):
        process = mp.Process(target=fourmi_proba_process, args=(i,resultat_queue,Graph,Pheromone,PositionFourmis,SeenFourmis,alpha,beta))
        process.start()
        processes.append(process)
    print("test1")
    for process in processes:
        process.join()
    print("test2")
    #On récupère les résultats
    for i in range(len(PositionFourmis)) :
        Result_temp,Distances_temp = resultat_queue.get()
        print("test3:",i)
        Result.append(Result_temp)
        #On calcul la distance totale pour normaliser les probabilités (on prend pas en compte les None)
        Distances.append(sum([element for element in Distances_temp if element != None]))

    #On normalise les probabilités
    for i in range(len(Result)) :
        for j in range(len(Result[i])) :
            if Result[i][j][2] != None :
                #On normalise les probabilités
                Result[i][j][2] = (Result[i][j][2]/Distances[i])
    return Result

def init_pheromone(Graph) : #Trustable
    #On initialise les phéromones
    Pheromone = []
    for i in range(len(Graph)) :
        Pheromone_temp = []
        for j in range(len(Graph)) :
            if i == j :
                Pheromone_temp.append(None)
            else :
                Pheromone_temp.append(1)
        Pheromone.append(Pheromone_temp)
    return Pheromone

def init_seen(NombreFourmis,graph): #Trustable
    #On initialise les fourmis
    Seen = []
    for i in range(NombreFourmis) :
        Seen_temp = []
        for j in range(len(graph)) :
            if i == j :
                Seen_temp.append(True)
            else :
                Seen_temp.append(False)
        Seen.append(Seen_temp)
    return Seen

def choix_fourmi(probas) : #Trustable
    Nombre_random = random.uniform(0, 1)
    Scaller = 0 # cette variable permet de stocker la somme des probabilités de chaque fourmis à chaque itération
    for proba in probas :
        if proba[2] != None :
            if Nombre_random <= Scaller + proba[2] :
                #On a trouvé le noeud de destination
                return proba[1]
            else :
                Scaller += proba[2]

def change_forme(Graph,Chemin,Start) :
    #Change la forme du chemin sous la forme (A,B,X)
    Result = []
    for node in Chemin :
        Result.append((Start,node,distance_euclidienne(Graph[Start],Graph[node])))
        Start = node
    return Result

def Pheromone_to_path(Graph,Pheromone,StartNode=0):
    #Result aura la forme d'une liste de tuple (A,B,X)
    Result = []
    Seen = [False if i != StartNode else True for i in range(len(Pheromone))] #Enpèche de boucler/revenir sur ses pas
    Node = StartNode
    for i in range(len(Pheromone)) :
        max = 1
        index_max = 0
        for index,nombre in enumerate(Pheromone[Node]) :
            #On filtre les cas indésirables
            if nombre == None or Seen[index]==True:
                continue
            if nombre > max:
                max = nombre
                index_max = index
        Result.append((Node,index_max,distance_euclidienne(Graph[Node],Graph[index_max])))
        Seen[index_max] = True
        Node = index_max

    #On génère le dernier lien
    Result.append((Node,StartNode,distance_euclidienne(Graph[Node],Graph[StartNode])))
        
    return Result
    
def maj_pheromone(pheromone,deplacementFourmis,distanceFourmis):
    nombreFourmis = len(deplacementFourmis)
    #On met à jour les phéromones
    for numero_fourmi in range(nombreFourmis) :
        for index_depart in range(len(deplacementFourmis[numero_fourmi])) :
            depart = deplacementFourmis[numero_fourmi][index_depart]
            arrive = deplacementFourmis[numero_fourmi][(index_depart+1)%len(deplacementFourmis[numero_fourmi])]
            #On actualise la matrice de phéromone
            pheromone[depart][arrive] += 1/distanceFourmis[numero_fourmi]
            pheromone[arrive][depart] += 1/distanceFourmis[numero_fourmi]
 
    return pheromone 

def facteur_alpha_beta(AlphaStart,BetaStart,AlphaEnd,BetaEnd,Iteration,NombreIteration):
    return AlphaStart + (AlphaEnd-AlphaStart)*Iteration/NombreIteration, BetaStart + (BetaEnd-BetaStart)*Iteration/NombreIteration




**Les Algorithmes**

In [3]:
###
### ALGO DE LA VERIF DE LA SOLUTION VERSION DECISIONNELLE DE TSP
###
def verif_graph_tsp(Graph,W):
    total = 0
    for element in Graph :
        total += element[2]
        if total >= W :
            return False
    return True


###
### ALGO Nearest Neighbor (NN) pour TSP
###
def nearest_neighbor(Graph):
    #Le graphe aura la forme d'un dictionnaire avec les coordonnées des noeux
    #On suppose que le graphe est connexe,complet,non orienté et pondéré
    #On suppose que les noeux sont numérotés de 0 à n-1
    Result = []
    Seen = [0]
    Index = 0
    while len(Result) != len(Graph) :
        Candidats = [] #Les candidats sont sous la forme (Noeud,Distance)
        for index_candidat in Graph :
            if index_candidat not in Seen :
                distance_candidat = distance_euclidienne(Graph[Index],Graph[index_candidat])
                Candidats.append((index_candidat,distance_candidat))
        if Candidats == [] :
            #On a fini le parcours
            Result.append((Index,0,distance_euclidienne(Graph[Index],Graph[0])))
            break
        Candidats.sort(key=lambda x:x[1])
        Result.append([Index,Candidats[0][0],Candidats[0][1]])
        Index = Candidats[0][0]
        Seen.append(Index)
    #On renvoie le chemin sous la forme d'une liste de noeux sous la forme d'une liste
    return Result

###
### ALGO FOURMI POUR TSP
###
def fourmi_tsp(Graph,NombreFourmis="graph",NombreIteration=100,Incidence = 1):
    #On suppose que le graphe est connexe,complet,non orienté et pondéré
    #On détermine le nombre de fourmis
    if NombreFourmis == "graph" :
        NombreFourmis = len(Graph)
    #On vérifie si il y a pas trop de fourmis par rapport au nombre de noeuds
    if NombreFourmis > len(Graph) :
        NombreFourmis = len(Graph)
    #On initialise les phéromones
    Pheromone = init_pheromone(Graph)
    for iteration in range(NombreIteration) :
        #chaque iteration correspond à un cycle de fourmis
        #Cette variable garde en mémoire les noeuds visiter par chaque fourmis
        Seen = init_seen(NombreFourmis,Graph)
        #Cette variable garde en mémoire la position de chaque fourmis (index = numéro de la fourmis et List[index] = position de la fourmis)
        NoeudsFourmis = [i for i in range(NombreFourmis)]
        #Cette variable enregistre les mouvements des fourmis
        DeplacementFourmis = []
        for i in range(NombreFourmis) :
            DeplacementFourmis.append([])
        #Cette variable enregistre les distances parcourues par les fourmis
        DistanceFourmis = [0]*NombreFourmis
        for i in range(len(Graph)) :
            #chaque iteration est un mouvement de toutes les fourmis
            #On calcul les probabilités de mouvement de chaque fourmis
            numero_fourmi = 0
            #On calcule les facteurs alpha et beta
            Alpha,Beta  = facteur_alpha_beta(Incidence,1/Incidence,1/Incidence,Incidence,iteration,NombreIteration)
            for pos_fourmi,probas in zip(NoeudsFourmis,fourmis_proba_main(Graph,Pheromone,NoeudsFourmis,Seen,alpha=Alpha,beta=Beta)) :
                #On vérifie le cas ou la fourmi doit revenir à son point de départ
                if probas == "F" :
                    Nouvelle_Position_Fourmi = numero_fourmi
                else :
                    #On choisit aléatoirement un noeud en fonction des probabilités
                    Nouvelle_Position_Fourmi = choix_fourmi(probas)
                #On calcule la distance entre les deux noeuds et on l'ajoute à la distance parcourue par la fourmis
                DistanceFourmis[numero_fourmi] += distance_euclidienne(Graph[pos_fourmi],Graph[Nouvelle_Position_Fourmi])
                #On actualise la position de la fourmis
                NoeudsFourmis[numero_fourmi] = Nouvelle_Position_Fourmi
                #On met à jour la table des Seens
                Seen[numero_fourmi][Nouvelle_Position_Fourmi] = True
                #On ajoute la position de la fourmis a la liste des déplacement
                DeplacementFourmis[numero_fourmi].append(Nouvelle_Position_Fourmi)
                #On change de fourmis
                numero_fourmi += 1
        #On met à jour les phéromones
        Pheromone = maj_pheromone(Pheromone,DeplacementFourmis,DistanceFourmis)
    return Pheromone_to_path(Graph,Pheromone,0)



Afficher un graphe

In [ ]:
graph = random_graph_complet(3000,10000)
print(graph)
test_nn = nearest_neighbor(graph)
plot_graph(graph,test_nn,False)

Tests de performance

In [ ]:
Temps = 0
Compteur = 2
Data_perf = []
while Temps<1:
    graph = random_graph_complet(Compteur)
    start = time.time()
    test_nn = nearest_neighbor(graph)
    end = time.time()
    Temps = end-start
    Data_perf.append((Compteur,Temps))
    Compteur += 1
plot_perf(Data_perf,True)

**Algo de la fourmi**

Tests

In [6]:
# Alpha : Importance des distances
# Beta : Importance des phéromones
# Gamma : Taux de changement entre Alpha et Beta
#Format du graphe : [(A,B,X),...] avec A : Noeux1 ; B : Noeux2 ; X : Poids de l'arête
graph = random_graph_complet(8)

# fourmi_tsp(graph,nombre_fourmis,nombre_iterations,alpha,beta)
data_final_test = fourmi_tsp(graph,"graph",1,2)
print("Méthode Fourmi")
plot_graph(graph,data_final_test)
print("PoidsGraph Fourmi :",PoidsGraph(data_final_test))


test_nn = nearest_neighbor(graph)
print("Méthode NN")
plot_graph(graph,test_nn)
print("PoidsGraph NN:",PoidsGraph(test_nn))




test1
test2


Mesure des performance

On suppose que les performances (en therme de vitesse de calcul) ne varie pas en focntion de alpha et beta

On cherche le meilleur combo (alpha,beta) afin d'avoir le meilleur resultat.

Impact de Alpha et Beta sur les performances de l'algorithme

In [ ]:
NombredeTests = 10 #Nombre de tests pour chaque couple (Alpha,Beta)
NombredeNoeuds = 50
NombreIterations = int(NombredeNoeuds**(1.5))
NombreFourmis = 1
AlphaStart = 0.1
BetaStart = 0.1
AlphaEnd = 2
BetaEnd = 2
PasAlpha = 0.5
PasBeta = 0.5
Data = [] #Format : [(Alpha,Beta,PoidsGraph),...]

IterationAlpha = int((AlphaEnd-AlphaStart)/PasAlpha)
IterationBeta = int((BetaEnd-BetaStart)/PasBeta)

for alpha_number in range(IterationAlpha) :
    for beta_number in range(IterationBeta) :
        PoidsGraphTotal = 0
        for i in range(NombredeTests) :
            AlphaTest = AlphaStart + alpha_number*PasAlpha
            BetaTest = BetaStart + beta_number*PasBeta
            graph = random_graph_complet(NombredeNoeuds)
            data_final_test = fourmi_tsp(graph,NombreFourmis,NombreIterations,AlphaTest,BetaTest)
            PoidsGraphTotal += PoidsGraph(data_final_test)
        Data.append((AlphaTest,BetaTest,PoidsGraphTotal/NombredeTests))
        #print("Alpha :",AlphaTest,"Beta :",BetaTest,"PoidsGraphMoyen :",PoidsGraphTotal/NombredeTests)
    print("Complété à ",(alpha_number+1)/IterationAlpha*100,"%")

def plot_perf_alpha_beta(Data,NombredeNoeuds,NombreIterations,NombreFourmis) :
    DataAlpha = []
    DataBeta = []
    DataPoids = []
    for alpha,beta,poids in Data :
        DataAlpha.append(alpha)
        DataBeta.append(beta)
        DataPoids.append(poids)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(DataAlpha,DataBeta,DataPoids)
    ax.set_xlabel('Alpha')
    ax.set_ylabel('Beta')
    ax.set_zlabel('Poids')
    plt.title("Nombre de noeuds : "+str(NombredeNoeuds)+" ; Nombre d'itérations : "+str(NombreIterations)+" ; Nombre de fourmis : "+str(NombreFourmis))
    plt.show()  
plot_perf_alpha_beta(Data,NombredeNoeuds,NombreIterations,NombreFourmis)

Conclusion : Alpha et Beta n'enfluence pas la solution trouvé.

Hypothèse : Le nombre d'itérations influence la solution trouvé

In [ ]:
NombredeNoeuds = 50
IterationDebut = 50
IterationFin = 5000
PasIteration = 50
NombreFourmis = 1
NombredeTests = 1
NombreIterations = int((IterationFin-IterationDebut)/PasIteration)
for numero_iteration in range(NombreIterations) :
    PoidsGraphTotal = 0
    for i in range(NombredeTests) :
        IterationTest = IterationDebut + numero_iteration*PasIteration
        graph = random_graph_complet(NombredeNoeuds)
        data_final_test = fourmi_tsp(graph,NombreFourmis,IterationTest,AlphaTest,BetaTest)
        PoidsGraphTotal += PoidsGraph(data_final_test)
    Data.append((IterationTest,PoidsGraphTotal/NombredeTests))
    print("Complété à",(numero_iteration+1)/NombreIterations*100,"%") 

def plot_perf_iteration(Data):
    DataIteration = []
    DataPoids = []
    for iteration,poids in Data :
        DataIteration.append(iteration)
        DataPoids.append(poids)
    plt.plot(DataIteration,DataPoids)
    plt.xlabel("Nombre d'itérations")
    plt.ylabel("Poids")
    plt.title("Nombre de noeuds : "+str(NombredeNoeuds)+" ; Nombre de fourmis : "+str(NombreFourmis))
    plt.show()
plot_perf_iteration(Data)

Conclusion : Le nombre d'itérations influence la qualité des solutions trouvé

Hypothèse : Alpha et Beta modifient la vitesse de convergence de l'algorithme

In [ ]:
NombredeTests = 10 #Nombre de tests pour chaque couple (Alpha,Beta)
NombredeNoeuds = 50
NombreIterations = NombredeNoeuds**(1.5)
NombreFourmis = 1
AlphaStart = 0.1
BetaStart = 0.1
AlphaEnd = 2
BetaEnd = 2
PasAlpha = 0.5
PasBeta = 0.5
Data = [] #Format : [(Alpha,Beta,PoidsGraph),...]

IterationAlpha = int((AlphaEnd-AlphaStart)/PasAlpha)
IterationBeta = int((BetaEnd-BetaStart)/PasBeta)

for alpha_number in range(IterationAlpha) :
    for beta_number in range(IterationBeta) :
        PoidsGraphTotal = 0
        for i in range(NombredeTests) :
            AlphaTest = AlphaStart + alpha_number*PasAlpha
            BetaTest = BetaStart + beta_number*PasBeta
            graph = random_graph_complet(NombredeNoeuds)
            test_pheromone = fourmi_tsp(graph,NombreFourmis,NombreIterations,AlphaTest,BetaTest)
            data_final_test = Pheromone_to_path(graph,test_pheromone,0)
            PoidsGraphTotal += PoidsGraph(data_final_test)
        Data.append((AlphaTest,BetaTest,PoidsGraphTotal/NombredeTests))
        #print("Alpha :",AlphaTest,"Beta :",BetaTest,"PoidsGraphMoyen :",PoidsGraphTotal/NombredeTests)
    print("Complété à :",(alpha_number+1)/IterationAlpha*100,"%")

def plot_perf_alpha_beta(Data,NombredeNoeuds,NombreIterations,NombreFourmis) :
    DataAlpha = []
    DataBeta = []
    DataPoids = []
    for alpha,beta,poids in Data :
        DataAlpha.append(alpha)
        DataBeta.append(beta)
        DataPoids.append(poids)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(DataAlpha,DataBeta,DataPoids)
    ax.set_xlabel('Alpha')
    ax.set_ylabel('Beta')
    ax.set_zlabel('Poids')
    plt.title("Nombre de noeuds : "+str(NombredeNoeuds)+" ; Nombre d'itérations : "+str(NombreIterations)+" ; Nombre de fourmis : "+str(NombreFourmis))
    plt.show()  
plot_perf_alpha_beta(Data,NombredeNoeuds,NombreIterations,NombreFourmis)